# Data Preparation

## Select Data - Eliminar Columnas
De la seccion de entendimiento de los datos, vimos que hay columnas que realmente no aportan mucha información, importancia o significancia al alcance de este proyecto, por lo que en esta parte, se eliminaran para contar unicamente con los datos que se trabajaran, estas columnas son:

- municipio_hechos : Al tener el 99% de sus datos como NaN y no aportar informacion
- competencia : Al tener un unico valor
- coordenadas (latitud y longitud) : El proyecto se enfocara en las alcaldias de la Ciudad de Mexico y el valor de coordenadas no estan dentro de
este scope
- fgj_colonia_registro : No esta dentro del scope de este proyecto

In [217]:
import pandas as pd

data_set = pd.read_csv("sourceData.csv")

In [218]:
columns_to_remove = ['municipio_hechos', 'competencia', 'latitud', 'longitud', 'fgj_colonia_registro']
data_set = data_set.drop(columns_to_remove, axis=1)

data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,colonia_datos
0,8324429.0,2019,Enero,04/01/2019,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,12:00:00,12:19:00,ALVARO OBREGON,GUADALUPE INN
1,8324430.0,2019,Enero,04/01/2019,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,15:00:00,12:20:00,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,Enero,04/01/2019,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,15:30:00,12:23:00,COYOACAN,COPILCO EL BAJO
3,8324435.0,2019,Enero,04/01/2019,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,06:00:00,12:27:00,IZTACALCO,PANTITLAN V
4,8324438.0,2019,Enero,04/01/2019,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,NaN,FISICA,VICTIMA,2019.0,Enero,03/01/2019,20:00:00,12:35:00,IZTAPALAPA,LAS AMERICAS (U HAB)


## Outliers
Las columnas que cuentan con valores outliers son:
- edad: Valores mayores a 120 (258, 361 y 369) y 1899 valores de 0
- año_hechos: Un valor de 222 y valores del siglo pasado

In [219]:
valores_edad = data_set.groupby('Edad')['idCarpeta'].count()
valores_ahecho = data_set.groupby('Anio_hecho')['idCarpeta'].count()

### Outliers - Edad
El approach a seguir es el obtener un promedio de edad entre un limite inferior (1) y un limite superior (120)
Despues de obtener este promedio, reemplazamos los outliers (valores de 0, 258, 361 y 369) con este promedio, ya que el numero de valores en 0 es significativo (1899)

In [220]:
limite_inferior = 1
limite_superior = 120
edad_en_rango = data_set.loc[data_set['Edad'].between(limite_inferior, limite_superior), 'Edad']
promedio_edad = edad_en_rango.mean()
print(promedio_edad)

promedio_edad = round(promedio_edad)
print(promedio_edad)

38.944485754445886
39


In [221]:
data_set['Edad'] = data_set['Edad'].mask(~data_set['Edad'].between(limite_inferior, limite_superior), promedio_edad)
print(data_set.groupby('Edad')['idCarpeta'].count())

Edad
1.0       942
2.0      1206
3.0      1685
4.0      1837
5.0      1848
         ... 
107.0       1
110.0       1
111.0       1
114.0       1
120.0       2
Name: idCarpeta, Length: 110, dtype: int64


## Outliers - Año Hechos
Se borran las filas con valores menores a 2000

In [222]:
data_set = data_set[data_set['Anio_hecho'] >= 2000]
print(data_set.groupby('Anio_hecho')['idCarpeta'].count())

Anio_hecho
2000.0       118
2001.0        93
2002.0       212
2003.0        68
2004.0        84
2005.0        84
2006.0       108
2007.0       129
2008.0       231
2009.0       254
2010.0       353
2011.0       430
2012.0       531
2013.0       541
2014.0       880
2015.0      1085
2016.0      1732
2017.0      3457
2018.0     17888
2019.0    261051
2020.0    214736
2021.0    235849
2022.0    233278
Name: idCarpeta, dtype: int64


## Missing Data
Las columnas con valores NaN son las siguientes:
- Sexo : 180,609 registros
- TipoPersona : 6512 registros
- CalidadJuridica: 1 registro
- Anio_hecho, Mes_hecho, FechaHecho : 370 registros
- HoraHecho : 361 registros
- HoraInicio : 1
- colonia_datos : 69,602 registros

In [223]:
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

Sexo : 180471
TipoPersona : 6502
CalidadJuridica : 1
HoraHecho : 1
HoraInicio : 1
colonia_datos : 69462


### Grupo 1 - Columnas con menos de 1000 registros Nan
El approach para este grupo es borrar los registros, ya que son de un numero muy bajo
Las columnas a borrar sus registros con valores nulos son:
- CalidadJuridica
- Anio_hecho
- Mes_hecho
- FechaHecho
- HoraHecho
- HoraInicio

In [224]:
data_set = data_set.dropna(subset=['CalidadJuridica', 'Anio_hecho', 'Mes_hecho', 'FechaHecho', 'HoraHecho', 'HoraInicio'])
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

Sexo : 180471
TipoPersona : 6502
colonia_datos : 69461


### Grupo 2 - Columnas con mas de 1000 registros Nan
Para estas columnas, el approach es sustituir los valores Nulos con la palabra 'Desconocido'
Las columnas a ser modificadas son:
- Sexo
- TipoPersona
- colonia_datos

In [225]:
data_set[['Sexo', 'TipoPersona', 'colonia_datos']] = data_set[['Sexo', 'TipoPersona', 'colonia_datos']].fillna('Desconocido')
for column in data_set.columns:
    if(data_set[column].isnull().sum() > 0):
        print(column + ' : ' + str(data_set[column].isnull().sum()))

## Data Transformation
En esta seccion, se modificaran las columnas de formato HH:MM:SS a HH:MM
Estas columnas son:
- HoraHecho
- HoraInicio

In [226]:
data_set[['HoraHecho', 'HoraInicio']] = data_set[['HoraHecho', 'HoraInicio']].apply(pd.to_datetime)
data_set['HoraHecho'] = data_set['HoraHecho'].dt.strftime('%H:%M')
data_set['HoraInicio'] = data_set['HoraInicio'].dt.strftime('%H:%M')
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,FechaInicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,colonia_datos
0,8324429.0,2019,Enero,04/01/2019,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,12:00,12:19,ALVARO OBREGON,GUADALUPE INN
1,8324430.0,2019,Enero,04/01/2019,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,15:00,12:20,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,Enero,04/01/2019,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,15:30,12:23,COYOACAN,COPILCO EL BAJO
3,8324435.0,2019,Enero,04/01/2019,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,06:00,12:27,IZTACALCO,PANTITLAN V
4,8324438.0,2019,Enero,04/01/2019,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019.0,Enero,03/01/2019,20:00,12:35,IZTAPALAPA,LAS AMERICAS (U HAB)


### Crear Columnas Dia y Mes
Se observo que contamos con dos columnas FechaInicio y FechaHecho, de igual forma para cada una de estas se cuenta con dos columnas Mes y Anio
Las columnas FechaInicio y FechaHecho son de tipo DD-MM-YYYY, las columnas Mes_inicio y Mes_hecho representan los valores MM de dichas columnas, y las columnas con prefijo Anio indican los valores YYYY respectivamente
Con el proposito de contar con datos mas limpios y entendibles, se seguira el siguiente algoritmo:
1. Crear una columna Dia_inicio que representara el valor numerico DD de la columna FechaInicio
2. Reemplazar los valores de Mes_inicio por su valor numerico MM de la columna FechaInicio
3. Eliminar la columna FechaInicio
4. Seguir los pasos 1-3 para la columna FechaHecho

In [227]:
# Fecha Inicio
# Dias
dias_inicio = pd.to_datetime(data_set['FechaInicio'], format='%d/%m/%Y').dt.day
data_set.insert(3, 'Dia_inicio', dias_inicio)
# Meses
meses_inicio = pd.to_datetime(data_set['FechaInicio'], format='%d/%m/%Y').dt.month
data_set['Mes_inicio'] = meses_inicio
# Año
anios_inicio = pd.to_datetime(data_set['FechaInicio'], format='%d/%m/%Y').dt.year
data_set['Anio_inicio'] = anios_inicio

data_set = data_set.drop('FechaInicio', axis=1)
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,Dia_inicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,FechaHecho,HoraHecho,HoraInicio,alcaldia_hechos,colonia_datos
0,8324429.0,2019,1,4,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018.0,Agosto,29/08/2018,12:00,12:19,ALVARO OBREGON,GUADALUPE INN
1,8324430.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,15/12/2018,15:00,12:20,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,1,4,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018.0,Diciembre,22/12/2018,15:30,12:23,COYOACAN,COPILCO EL BAJO
3,8324435.0,2019,1,4,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019.0,Enero,04/01/2019,06:00,12:27,IZTACALCO,PANTITLAN V
4,8324438.0,2019,1,4,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019.0,Enero,03/01/2019,20:00,12:35,IZTAPALAPA,LAS AMERICAS (U HAB)


In [228]:
# Fecha Hecho
# Dias
dias_hecho = pd.to_datetime(data_set['FechaHecho'], format='%d/%m/%Y').dt.day
data_set.insert(12, 'Dia_hecho', dias_hecho)
# Meses
meses_hecho = pd.to_datetime(data_set['FechaHecho'], format='%d/%m/%Y').dt.month
data_set['Mes_hecho'] = meses_hecho
# Año
anios_hecho = pd.to_datetime(data_set['FechaHecho'], format='%d/%m/%Y').dt.year
data_set['Anio_hecho'] = anios_hecho

data_set = data_set.drop('FechaHecho', axis=1)
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,Dia_inicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,Dia_hecho,HoraHecho,HoraInicio,alcaldia_hechos,colonia_datos
0,8324429.0,2019,1,4,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018,8,29,12:00,12:19,ALVARO OBREGON,GUADALUPE INN
1,8324430.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,15,15:00,12:20,AZCAPOTZALCO,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,1,4,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,22,15:30,12:23,COYOACAN,COPILCO EL BAJO
3,8324435.0,2019,1,4,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019,1,4,06:00,12:27,IZTACALCO,PANTITLAN V
4,8324438.0,2019,1,4,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019,1,3,20:00,12:35,IZTAPALAPA,LAS AMERICAS (U HAB)


## Data Integration

### Poblacion Alcaldias
En esta fase nos centraremos en agregar información externa proveniente de otras bases de datos y fuentes de información
Para esto, integraremos datos obtenidos de las estadisticas de Poblacion del INEGI para mostrar la poblacion por alcaldia, agregando una nueva columna de poblacion dependiendo el valor de la columna de alcaldia_hechos
Para realizar este mapeo entre la alcaldia de los hechos y su poblacion, se llevo a cabo un script en Python

In [229]:
from mapeo_poblacion import get_population

data_set.insert(loc=16, column='poblacion_alcaldia', value=data_set['alcaldia_hechos'].apply(get_population))
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,Dia_inicio,Delito,Categoria,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,Dia_hecho,HoraHecho,HoraInicio,alcaldia_hechos,poblacion_alcaldia,colonia_datos
0,8324429.0,2019,1,4,FRAUDE,DELITO DE BAJO IMPACTO,Masculino,62.0,FISICA,OFENDIDO,2018,8,29,12:00,12:19,ALVARO OBREGON,759137,GUADALUPE INN
1,8324430.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",DELITO DE BAJO IMPACTO,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,15,15:00,12:20,AZCAPOTZALCO,432205,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,1,4,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,ROBO A CUENTAHABIENTE SALIENDO DEL CAJERO CON ...,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,22,15:30,12:23,COYOACAN,614447,COPILCO EL BAJO
3,8324435.0,2019,1,4,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019,1,4,06:00,12:27,IZTACALCO,404695,PANTITLAN V
4,8324438.0,2019,1,4,ROBO DE MOTOCICLETA SIN VIOLENCIA,ROBO DE VEHÍCULO CON Y SIN VIOLENCIA,Masculino,39.0,FISICA,VICTIMA,2019,1,3,20:00,12:35,IZTAPALAPA,1835486,LAS AMERICAS (U HAB)


### Categorización de Delitos
Actualmente el dataset cuenta con 16 tipos diferentes de Categorias y 310 tipos diferentes de Delitos, al ser tantos valores unicos se dificulta el
analisis conciso de este tipo de delitos y los 16 tipos de categoria son algo ambigüos como "Delito de bajo Impacto".

Para esto se ha utilizado de un algoritmo open source que emplea técnicas avanzadas de procesamiento de lenguaje natural, para clasificar distintos
tipos de delitos en funcion de su nivel de riesgo y severidad. Este algoritmo analiza y evalua la gravedad de cada delito, permitiendo establecer
rangos que van desde situaciones de menor riesgo (valor de 1) hasta aquellas que presentan un mayor nivel de peligro (valor de 10). El objetivo de esta
herramienta es proporcionar una clasificacion precisa y confiable de los delitos.

De esta forma la columna 'Categoria' deja de ser util y puede ser eliminada.

In [230]:
numero_categorias = len(pd.unique(data_set['Categoria']))
numero_delitos = len(pd.unique(data_set['Delito']))
print(f"Categorias Distintas =  {numero_categorias}; Delitos Distintos =  {numero_delitos}")

Categorias Distintas =  16; Delitos Distintos =  310


In [231]:
from mapeo_crimen import get_crime_score

data_set.insert(loc=5, column='riesgo_delito', value=data_set['Delito'].apply(get_crime_score))
data_set = data_set.drop('Categoria', axis=1)
data_set.head()

,idCarpeta,Anio_inicio,Mes_inicio,Dia_inicio,Delito,riesgo_delito,Sexo,Edad,TipoPersona,CalidadJuridica,Anio_hecho,Mes_hecho,Dia_hecho,HoraHecho,HoraInicio,alcaldia_hechos,poblacion_alcaldia,colonia_datos
0,8324429.0,2019,1,4,FRAUDE,5,Masculino,62.0,FISICA,OFENDIDO,2018,8,29,12:00,12:19,ALVARO OBREGON,759137,GUADALUPE INN
1,8324430.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",8,Femenino,38.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,15,15:00,12:20,AZCAPOTZALCO,432205,VICTORIA DE LAS DEMOCRACIAS
2,8324431.0,2019,1,4,ROBO A TRANSEUNTE SALIENDO DEL BANCO CON VIOLE...,10,Masculino,42.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,22,15:30,12:23,COYOACAN,614447,COPILCO EL BAJO
3,8324435.0,2019,1,4,ROBO DE VEHICULO DE SERVICIO PARTICULAR SIN VI...,3,Masculino,35.0,FISICA,VICTIMA Y DENUNCIANTE,2019,1,4,06:00,12:27,IZTACALCO,404695,PANTITLAN V
4,8324438.0,2019,1,4,ROBO DE MOTOCICLETA SIN VIOLENCIA,3,Masculino,39.0,FISICA,VICTIMA,2019,1,3,20:00,12:35,IZTAPALAPA,1835486,LAS AMERICAS (U HAB)
5,8324442.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",8,Femenino,42.0,FISICA,OFENDIDO,2018,10,12,18:00,12:38,COYOACAN,614447,LOS REYES (PBLO)
6,8324444.0,2019,1,4,ROBO A TRANSEUNTE DE CELULAR SIN VIOLENCIA,2,Femenino,55.0,FISICA,VICTIMA Y DENUNCIANTE,2019,1,4,08:30,12:40,ALVARO OBREGON,759137,TOLTECA
7,8324451.0,2019,1,4,VIOLACION,10,Masculino,13.0,FISICA,VICTIMA,2010,12,19,03:00,12:48,GUSTAVO A. MADERO,1173351,GENERAL FELIPE BERRIOZABAL
8,8324454.0,2019,1,4,"PRODUCCIÓN, IMPRESIÓN, ENAJENACIÓN, DISTRIBUCI...",8,Femenino,83.0,FISICA,VICTIMA Y DENUNCIANTE,2018,12,27,13:00,12:51,IZTAPALAPA,1835486,BANJIDAL
9,8324455.0,2019,1,4,OMISION DE AUXILIO O DE CUIDADO,1,Masculino,15.0,FISICA,VICTIMA,2018,1,4,13:00,12:51,VENUSTIANO CARRANZA,443704,Desconocido


In [232]:
data_set.to_csv('data_preparation.csv', index=False)